Everything to Get Current Season

Scrape

In [13]:
import pandas as pd
import csv
import os
from pathlib import Path

# Set working directory to project root if run from notebooks/
if Path.cwd().name == "notebooks":
    os.chdir("..")

# Define folders – assume they already exist
raw_data_dir = Path("data/Raw")
cleaned_data_dir = Path("data/Cleaned")
metrics_data_dir = Path("data/Metrics")
current_season_dir = metrics_data_dir / "CurrentSeasonTeams"

url = 'https://www.basketball-reference.com/wnba/years/2025_totals.html'
df2025 = pd.read_html(url)[0]  # First table is the player totals
df2025.to_csv(raw_data_dir / '2025Data.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')
df2025 = df2025[df2025['Player'] != 'Player']  # Drop header rows repeated in table
df2025 = df2025.drop_duplicates()        # Remove duplicate entries, if any
tot_players = df2025[df2025['Team'] == 'TOT']['Player']
df2025 = pd.concat([
    df2025[df2025['Team'] == 'TOT'],                              # all TOT rows
    df2025[~df2025['Player'].isin(tot_players) & (df2025['Team'] != 'TOT')]  # single-team players
], ignore_index=True)
df2025['MP'] = pd.to_numeric(df2025['MP'])
df2025 = df2025.drop('G.1', axis=1)
df2025 = df2025.drop('MP.1', axis=1)

cols_to_exclude = ['Player', 'Team', 'Pos']
cols_to_convert = df2025.columns.difference(cols_to_exclude)

df2025[cols_to_convert] = df2025[cols_to_convert].apply(pd.to_numeric, errors='coerce')

df2025

,Player,Team,Pos,G,MP,GS,FG,FGA,FG%,3P,...,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Grace Berger,TOT,G,8,141,2,7,33,0.212,0,...,4,0.000,4,21,15,6,1,10,18,14
1,Chloe Bibby,TOT,F,13,147,0,21,53,0.396,17,...,9,0.889,5,24,7,8,1,7,13,67
2,DeWanna Bonner,TOT,G-F,21,482,4,64,156,0.410,25,...,62,0.871,18,90,26,16,4,17,25,207
3,DiJonai Carrington,TOT,G-F,23,568,13,89,233,0.382,18,...,66,0.788,44,114,41,27,11,54,59,248
4,Kaila Charles,TOT,G-F,23,355,4,41,103,0.398,15,...,31,0.839,36,88,22,13,5,22,42,123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Gabby Williams,SEA,F,33,1077,33,159,362,0.439,41,...,61,0.852,19,151,149,82,14,64,47,411
170,Kiana Williams,PHO,G,6,69,0,12,24,0.500,5,...,2,0.500,0,7,7,1,0,1,2,30
171,A'ja Wilson,LVA,C,29,888,29,224,465,0.482,11,...,211,0.848,64,284,98,48,61,69,61,638
172,Jackie Young,LVA,G,33,990,33,193,422,0.457,54,...,132,0.909,41,139,151,44,10,79,87,560


Adding DRB

In [14]:
df2025['DRB'] = df2025['TRB'] - df2025['ORB']

# Define target and reference columns
col_to_move = 'DRB'
before_col = 'TRB'

# Get current column order and remove the column to move
cols = df2025.columns.tolist()
cols.remove(col_to_move)

# Find index to insert the column before the target
insert_at = cols.index(before_col)

# Insert the column at the desired position
cols.insert(insert_at, col_to_move)

# Reorder the DataFrame
df2025 = df2025[cols]

df2025.to_csv(cleaned_data_dir / '2025Data.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Metrics

In [15]:
df2025 = pd.read_csv(cleaned_data_dir / '2025Data.csv')

df2025['Efficiency'] = (1/df2025['G']) * (df2025['PTS'] + df2025['TRB'] + df2025['AST'] + df2025['STL'] - df2025['TOV'] - (df2025['FGA'] - df2025['FG']) - (df2025['FTA'] - df2025['FT']))

df2025['WinScore'] = (1/df2025['G']) * (df2025['PTS'] + df2025['TRB'] + df2025['STL'] + (df2025['AST'] * 0.5) + (df2025['BLK'] * 0.5) - df2025['FGA'] - df2025['TOV'] - (df2025['FTA'] * 0.5) - (df2025['PF'] * 0.5))

df2025['NBAPER'] = .68 + ((1/df2025['MP']) * ((26.77 * df2025['FG']) - (35.58 * df2025['FGA']) + (26.32 * df2025['FT']) - (25.47 * df2025['FTA']) + (42.74 * df2025['ORB']) + (11.45 * df2025['DRB']) + (29.7 * df2025['AST']) + (47.3 * df2025['STL']) + (38 * df2025['BLK']) - (47.9 * df2025['TOV']) - (18.8 * df2025['PF']) + (41.1 * df2025['PTS'])))

df2025['WNBAPER'] = .3289 + ((1/df2025['MP']) * ((53.35 * df2025['FG']) - (29.28 * df2025['FGA']) + (15.27 * df2025['FT']) - (25.34 * df2025['FTA']) + (47.72 * df2025['ORB']) + (10.95 * df2025['DRB']) + (31.61 * df2025['AST']) + (50.72 * df2025['STL']) + (50.68 * df2025['BLK']) - (45.88 * df2025['TOV']) - (61.23 * df2025['PF']) + (41.1 * df2025['PTS'])))

print("Columns before filtering (2025):", df2025.columns.tolist())

expected_cols = ['Player', 'Team', 'Pos', 'WinScore', 'Efficiency', 'NBAPER', 'WNBAPER']

df2025 = df2025[[col for col in expected_cols if col in df2025.columns]]

df2025.to_csv(current_season_dir / '2025Metrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Columns before filtering (2025): ['Player', 'Team', 'Pos', 'G', 'MP', 'GS', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Efficiency', 'WinScore', 'NBAPER', 'WNBAPER']


Teamless

In [16]:
df2025 = pd.read_csv(current_season_dir / '2025Metrics.csv')

df2025teamless = df2025.drop(columns=['Team'])

df2025teamless = df2025teamless.round(3)

df2025teamless.to_csv(metrics_data_dir / '2025Metrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Team Specific

In [17]:
df2025 = pd.read_csv(current_season_dir / '2025Metrics.csv')

df2025 = df2025.round(3)

# The column to split on
split_column = "Team"

# Directory to save files (ensure it exists or create it)
output_dir = current_season_dir

# Split and save
for group, group_df in df2025.groupby(split_column):
    filename = f"{output_dir}/{group}.csv"
    group_df = group_df.drop(columns=['Team'])  # Drop 'Team' column
    group_df.to_csv(filename, index=False, sep=',', encoding='utf-8', lineterminator='\n')

Combined DFs

In [18]:

df2023 = pd.read_csv(metrics_data_dir / '2023Metrics.csv')

df2024 = pd.read_csv(metrics_data_dir / '2024Metrics.csv')

df2025 = pd.read_csv(metrics_data_dir / '2025Metrics.csv')

combined = pd.concat([
    df2023.assign(Season=2023),
    df2024.assign(Season=2024),
    df2025.assign(Season=2025)
])

combined[['Player', 'Season', 'Pos', 'WinScore', 'Efficiency', 'NBAPER', 'WNBAPER']].to_csv(metrics_data_dir / 'AllPlayersMetrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')